In [35]:
import pandas as pd
df2=pd.read_csv('studentAssessment.csv')

In [36]:
df1=df2.groupby(['id_student','date_submitted'],as_index=False)['score'].sum()
df1.sample(5)

,id_student,date_submitted,score
82634,588219,24,58.0
108619,623295,39,49.0
141715,695916,256,67.0
27927,426871,79,94.0
72835,573864,171,22.0


In [34]:
df1[df1['score']>100]

,id_student,date_submitted,score
5,8462,-1,346.0
42,24186,207,200.0
99,26211,37,197.0
131,27189,138,202.0
133,27189,226,220.0
...,...,...,...
160454,2697921,220,215.0
160456,2697921,227,124.0
160457,2697921,231,303.0
160465,2698125,223,243.0


In [17]:
df3=df2.groupby(['id_student'],as_index=False)['score'].sum()
df3

,id_student,score
0,6516,309.0
1,8462,609.0
2,11391,410.0
3,23629,330.0
4,23698,670.0
...,...,...
23364,2698251,407.0
23365,2698257,339.0
23366,2698535,314.0
23367,2698577,322.0


In [18]:
#Calculating total number of assessments per student
total_assessments = df2.groupby('id_student')['id_assessment'].count().reset_index()
total_assessments

,id_student,id_assessment
0,6516,5
1,8462,7
2,11391,5
3,23629,4
4,23698,9
...,...,...
23364,2698251,7
23365,2698257,5
23366,2698535,8
23367,2698577,5


In [19]:
total_assessments.rename(columns={'id_assessment': 'total_assessments'}, inplace=True)
df2 = pd.merge(df2, total_assessments, on='id_student')


In [20]:
df2

,id_assessment,id_student,date_submitted,is_banked,score,total_assessments
0,1752,11391,18,0,78.0,5
1,1752,28400,22,0,70.0,5
2,1752,31604,17,0,72.0,5
3,1752,32885,26,0,69.0,5
4,1752,38053,19,0,79.0,5
...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,9
173908,37443,534672,229,0,100.0,7
173909,37443,546286,215,0,80.0,8
173910,37443,546724,230,0,100.0,9


In [21]:
# Step 2: Calculate completion percentage
df2['completion_percentage'] = (df2.groupby('id_student').cumcount() + 1) / df2['total_assessments'] * 100


In [22]:
def calculate_rs(student_df):
    rs = {}
    for threshold in [20, 40, 60, 80, 100]:
        rs[f'RS{threshold}'] = student_df[student_df['completion_percentage'] <= threshold]['score'].mean()
    return pd.Series(rs)


In [23]:
rs_scores = df2.groupby('id_student').apply(calculate_rs).reset_index()

print(rs_scores)

       id_student   RS20       RS40       RS60       RS80      RS100
0            6516   60.0  54.000000  57.000000  58.000000  61.800000
1            8462   93.0  88.000000  89.000000  87.800000  87.000000
2           11391   78.0  81.500000  81.000000  82.000000  82.000000
3           23629    NaN  67.000000  65.000000  76.666667  82.500000
4           23698   94.0  74.333333  73.600000  76.285714  74.444444
...           ...    ...        ...        ...        ...        ...
23364     2698251   69.0  72.500000  66.250000  65.400000  58.142857
23365     2698257   58.0  61.000000  66.000000  66.250000  67.800000
23366     2698535   23.0  28.000000  24.250000  36.000000  39.250000
23367     2698577  100.0  77.000000  70.000000  67.000000  64.400000
23368     2698588  100.0  92.500000  90.666667  91.750000  92.400000

[23369 rows x 6 columns]


C:\Users\jaina\AppData\Local\Temp\ipykernel_18792\1828764318.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  rs_scores = df2.groupby('id_student').apply(calculate_rs).reset_index()


In [11]:
student_assessment=df2

In [12]:
rs_scores.isnull().sum()

id_student       0
RS20          6725
RS40          3420
RS60          1968
RS80          1962
RS100           18
dtype: int64

In [14]:
rs_scores[['RS20', 'RS40', 'RS60', 'RS80', 'RS100']].nunique()


RS20      366
RS40      934
RS60     1600
RS80     2229
RS100    3187
dtype: int64

In [18]:
merged_df = pd.merge(student_assessment,rs_scores, on=["id_student"], how="inner")
merged_df.shape

(173912, 12)

In [19]:
stuassessment=merged_df.drop(['is_banked','total_assessments','completion_percentage'],axis=1)

In [20]:
stuassessment

,id_assessment,id_student,date_submitted,score,RS20,RS40,RS60,RS80,RS100
0,1752,11391,18,78.0,78.0,81.500000,81.000000,82.000000,82.000000
1,1752,28400,22,70.0,70.0,69.000000,69.333333,68.000000,66.400000
2,1752,31604,17,72.0,72.0,71.500000,72.333333,76.250000,76.000000
3,1752,32885,26,69.0,69.0,49.500000,54.000000,49.250000,54.400000
4,1752,38053,19,79.0,79.0,74.000000,74.000000,68.000000,68.000000
...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,60.0,65.0,61.666667,73.000000,77.857143,73.888889
173908,37443,534672,229,100.0,70.0,70.000000,79.750000,79.800000,79.857143
173909,37443,546286,215,80.0,69.0,79.333333,84.500000,89.666667,87.250000
173910,37443,546724,230,100.0,75.0,63.666667,78.200000,84.428571,81.222222


In [17]:
stuassessment.to_csv("stuassessment.csv", index=False)


In [18]:
stuassessment.isnull().sum()

id_assessment         0
id_student            0
date_submitted        0
score               173
RS20              16830
RS40               4896
RS60               1989
RS80               1971
RS100                19
dtype: int64

In [19]:
stuassessment.shape

(173912, 9)

In [21]:
stuassessment['id_student'].nunique()

23369